# PREVENDO INTENÇÃO DE COMPRA (SVM)

> *O conjunto de dados consiste em vetores de recursos pertencentes a 12.330 sessões. O conjunto de dados foi formado de modo que cada sessão pertencesse a um usuário diferente em um período de 1 ano, para evitar a influência de qualquer tendência para uma campanha específica, dia especial, perfil de usuário ou período específico. O conjunto de dados consiste em 10 atributos numéricos e 8 categóricos. O atributo 'Revenue' pode ser usado como o rótulo da classe.*]

<br>

**OBJETIVO**

**-Prever intenção de compra de usuários de comércio eletrônico.**

<br>

**Dataset:** https://archive.ics.uci.edu/ml/datasets/Online+Shoppers+Purchasing+Intention+Dataset

## CARREGANDO PACOTES E IMPORTANDO DADOS

In [1]:
# Carregando pacotes
library(tidyverse)
library(caret)
library(gains)
library(pROC)
library(ROCR)
library(ROSE)
library(e1071)
library(mice)
library(DMwR)
library(multiROC)
library(knitr)
library(gridExtra)

Warning message:
"package 'tidyverse' was built under R version 3.6.3"-- Attaching packages --------------------------------------- tidyverse 1.3.0 --
v ggplot2 3.3.3     v purrr   0.3.4
v tibble  3.0.6     v dplyr   1.0.4
v tidyr   1.1.2     v stringr 1.4.0
v readr   1.3.1     v forcats 0.4.0
Warning message:
"package 'ggplot2' was built under R version 3.6.3"Warning message:
"package 'tibble' was built under R version 3.6.3"Warning message:
"package 'tidyr' was built under R version 3.6.3"Warning message:
"package 'purrr' was built under R version 3.6.3"Warning message:
"package 'dplyr' was built under R version 3.6.3"-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
Warning message:
"package 'caret' was built under R version 3.6.3"Loading required package: lattice

Attaching package: 'caret'

The following object is masked from 'package:purrr':

    lift

Warning message:
"pac

In [2]:
# Importando os dados
df_shoppers<- read.csv("online_shoppers_intention.csv")

In [3]:
# Sumário dos dados
summary(df_shoppers)

 Administrative   Administrative_Duration Informational   
 Min.   : 0.000   Min.   :  -1.00         Min.   : 0.000  
 1st Qu.: 0.000   1st Qu.:   0.00         1st Qu.: 0.000  
 Median : 1.000   Median :   8.00         Median : 0.000  
 Mean   : 2.318   Mean   :  80.91         Mean   : 0.504  
 3rd Qu.: 4.000   3rd Qu.:  93.50         3rd Qu.: 0.000  
 Max.   :27.000   Max.   :3398.75         Max.   :24.000  
 NA's   :14       NA's   :14              NA's   :14      
 Informational_Duration ProductRelated   ProductRelated_Duration
 Min.   :  -1.00        Min.   :  0.00   Min.   :   -1.0        
 1st Qu.:   0.00        1st Qu.:  7.00   1st Qu.:  185.0        
 Median :   0.00        Median : 18.00   Median :  599.8        
 Mean   :  34.51        Mean   : 31.76   Mean   : 1196.0        
 3rd Qu.:   0.00        3rd Qu.: 38.00   3rd Qu.: 1466.5        
 Max.   :2549.38        Max.   :705.00   Max.   :63973.5        
 NA's   :14             NA's   :14       NA's   :14             
  Bounce

In [4]:
# Visualizar as seis primeiras linhas do dataset
head(df_shoppers)

Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0,0,1,0.000000,0.20000000,0.2000000,0,0,Feb,1,1,1,1,Returning_Visitor,FALSE,FALSE
0,0,0,0,2,64.000000,0.00000000,0.1000000,0,0,Feb,2,2,1,2,Returning_Visitor,FALSE,FALSE
0,-1,0,-1,1,-1.000000,0.20000000,0.2000000,0,0,Feb,4,1,9,3,Returning_Visitor,FALSE,FALSE
0,0,0,0,2,2.666667,0.05000000,0.1400000,0,0,Feb,3,2,2,4,Returning_Visitor,FALSE,FALSE
0,0,0,0,10,627.500000,0.02000000,0.0500000,0,0,Feb,3,3,1,4,Returning_Visitor,TRUE,FALSE
0,0,0,0,19,154.216667,0.01578947,0.0245614,0,0,Feb,2,2,1,3,Returning_Visitor,FALSE,FALSE


In [5]:
# Visualizar as seis últimas linhas do dataset
tail(df_shoppers)

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
12325,0,0,1,0,16,503.000,0.000000000,0.03764706,0.00000,0,Nov,2,2,1,1,Returning_Visitor,FALSE,FALSE
12326,3,145,0,0,53,1783.792,0.007142857,0.02903061,12.24172,0,Dec,4,6,1,1,Returning_Visitor,TRUE,FALSE
12327,0,0,0,0,5,465.750,0.000000000,0.02133333,0.00000,0,Nov,3,2,1,8,Returning_Visitor,TRUE,FALSE
12328,0,0,0,0,6,184.250,0.083333333,0.08666667,0.00000,0,Nov,3,2,1,13,Returning_Visitor,TRUE,FALSE
12329,4,75,0,0,15,346.000,0.000000000,0.02105263,0.00000,0,Nov,2,2,3,11,Returning_Visitor,FALSE,FALSE
12330,0,0,0,0,3,21.250,0.000000000,0.06666667,0.00000,0,Nov,3,2,1,2,New_Visitor,TRUE,FALSE


In [6]:
# Visão geral dos dados
glimpse(df_shoppers)

Rows: 12,330
Columns: 18
$ Administrative          <int> 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2~
$ Administrative_Duration <dbl> 0, 0, -1, 0, 0, 0, -1, -1, 0, 0, 0, 0, 0, 0, 0~
$ Informational           <int> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0~
$ Informational_Duration  <dbl> 0, 0, -1, 0, 0, 0, -1, -1, 0, 0, 0, 0, 0, 0, 0~
$ ProductRelated          <int> 1, 2, 1, 2, 10, 19, 1, 1, 2, 3, 3, 16, 7, 6, 2~
$ ProductRelated_Duration <dbl> 0.000000, 64.000000, -1.000000, 2.666667, 627.~
$ BounceRates             <dbl> 0.200000000, 0.000000000, 0.200000000, 0.05000~
$ ExitRates               <dbl> 0.200000000, 0.100000000, 0.200000000, 0.14000~
$ PageValues              <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0~
$ SpecialDay              <dbl> 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4, 0.0, 0.8, 0~
$ Month                   <fct> Feb, Feb, Feb, Feb, Feb, Feb, Feb, Feb, Feb, F~
$ OperatingSystems        <int> 1, 2, 4, 3, 3, 2, 2, 1, 2, 2, 1, 1, 1, 2, 3, 1~
$ Browser      

In [7]:
# Checar NAs
sapply(df_shoppers,function(x) sum(is.na(x)))

Administrative Administrative_Duration           Informational 
                     14                      14                      14 
 Informational_Duration          ProductRelated ProductRelated_Duration 
                     14                      14                      14 
            BounceRates               ExitRates              PageValues 
                     14                      14                       0 
             SpecialDay                   Month        OperatingSystems 
                      0                       0                       0 
                Browser                  Region             TrafficType 
                      0                       0                       0 
            VisitorType                 Weekend                 Revenue 
                      0                       0                       0

## 2. TRANSFORMAÇÃO DOS DADOS

In [8]:
# Tratando NAs
df_shoppers<- na.omit(df_shoppers)

In [9]:
# Checando NAs
sapply(df_shoppers, function(x) sum(is.na(x)))

Administrative Administrative_Duration           Informational 
                      0                       0                       0 
 Informational_Duration          ProductRelated ProductRelated_Duration 
                      0                       0                       0 
            BounceRates               ExitRates              PageValues 
                      0                       0                       0 
             SpecialDay                   Month        OperatingSystems 
                      0                       0                       0 
                Browser                  Region             TrafficType 
                      0                       0                       0 
            VisitorType                 Weekend                 Revenue 
                      0                       0                       0

In [10]:
# Transformando variável Month e VisitorType em numérica
encode_ordinal <- function(x, order = unique(x)) {
  x <- as.numeric(factor(x, levels = order, exclude = NULL))
  x
}

In [11]:
# Aplicando enconding 
df_shoppers$Month<- encode_ordinal(df_shoppers$Month)
df_shoppers$VisitorType<- encode_ordinal(df_shoppers$VisitorType)

In [12]:
# Transformando variáveis categóricas em fatores
to.factors <- function(df, variables){
  for (variable in variables){
    df[[variable]] <- as.factor(paste(df[[variable]]))
  }
  return(df)
}

In [13]:
# Selecionando as variáveis categóricas
categorical.vars <- c('Administrative','Informational','SpecialDay','Month',
                      'OperatingSystems','Browser','Region','TrafficType',
                      'VisitorType','Weekend','Revenue')

In [14]:
# Fatorizando as variáveis categóricas
df_shoppers <- to.factors(df = df_shoppers, variables = categorical.vars)


In [15]:
# Visualizando os dados
glimpse(df_shoppers)

Rows: 12,316
Columns: 18
$ Administrative          <fct> 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2~
$ Administrative_Duration <dbl> 0, 0, -1, 0, 0, 0, -1, -1, 0, 0, 0, 0, 0, 0, 0~
$ Informational           <fct> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0~
$ Informational_Duration  <dbl> 0, 0, -1, 0, 0, 0, -1, -1, 0, 0, 0, 0, 0, 0, 0~
$ ProductRelated          <int> 1, 2, 1, 2, 10, 19, 1, 1, 2, 3, 3, 16, 7, 6, 2~
$ ProductRelated_Duration <dbl> 0.000000, 64.000000, -1.000000, 2.666667, 627.~
$ BounceRates             <dbl> 0.200000000, 0.000000000, 0.200000000, 0.05000~
$ ExitRates               <dbl> 0.200000000, 0.100000000, 0.200000000, 0.14000~
$ PageValues              <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0~
$ SpecialDay              <fct> 0, 0, 0, 0, 0, 0, 0.4, 0, 0.8, 0.4, 0, 0.4, 0,~
$ Month                   <fct> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1~
$ OperatingSystems        <fct> 1, 2, 4, 3, 3, 2, 2, 1, 2, 2, 1, 1, 1, 2, 3, 1~
$ Browser      

## 3. MODELO MACHINE LEARNING

In [16]:
# Split do dataset em treino e teste
indxTrain <- createDataPartition(y = df_shoppers$Revenue, p = 0.75, list = FALSE)
dados_treino <- df_shoppers[indxTrain,]
dados_teste <- df_shoppers[-indxTrain,]

In [17]:
# Checando o balanceamento de classe da variável dependente
prop.table(table(dados_treino$Revenue)) * 100


   FALSE     TRUE 
84.50796 15.49204 

In [18]:
# Aplicando balanceamento
dados_treino_balanceados <- SMOTE(Revenue ~ ., dados_treino, perc.over = 3000, perc.under = 200)

### Modelo 1

In [19]:
# Construindo modelo 1
modelo_v1 <- svm(Revenue ~ ., data = dados_treino)

In [20]:
# Sumário do modelo 1
summary(modelo_v1)


Call:
svm(formula = Revenue ~ ., data = dados_treino)


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  radial 
       cost:  1 

Number of Support Vectors:  2306

 ( 1201 1105 )


Number of Classes:  2 

Levels: 
 FALSE TRUE




In [21]:
# Fazendo previsões
previsoes_v1 <- predict(modelo_v1, dados_teste)

In [22]:
# Matriz de confusão
print(resultados_v1<-confusionMatrix(previsoes_v1, dados_teste$Revenue))

Confusion Matrix and Statistics

          Reference
Prediction FALSE TRUE
     FALSE  2517  279
     TRUE     85  198
                                         
               Accuracy : 0.8818         
                 95% CI : (0.8698, 0.893)
    No Information Rate : 0.8451         
    P-Value [Acc > NIR] : 3.331e-09      
                                         
                  Kappa : 0.4586         
                                         
 Mcnemar's Test P-Value : < 2.2e-16      
                                         
            Sensitivity : 0.9673         
            Specificity : 0.4151         
         Pos Pred Value : 0.9002         
         Neg Pred Value : 0.6996         
             Prevalence : 0.8451         
         Detection Rate : 0.8175         
   Detection Prevalence : 0.9081         
      Balanced Accuracy : 0.6912         
                                         
       'Positive' Class : FALSE          
                                         

In [23]:
# Score AUC modelo 1
curva_roc1 <- multiclass.roc(response = dados_teste$Revenue, predictor = as.numeric(as.factor(previsoes_v1)))
curva_roc1$auc

Setting direction: controls < cases


Multi-class area under the curve: 0.6912

### Modelo 2

In [24]:
# Construindo modelo 2
modelo_v2 <- svm(Revenue ~ ., data = dados_treino_balanceados)

In [25]:
# Sumário modelo 2
summary(modelo_v2)


Call:
svm(formula = Revenue ~ ., data = dados_treino_balanceados)


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  radial 
       cost:  1 

Number of Support Vectors:  34365

 ( 17352 17013 )


Number of Classes:  2 

Levels: 
 FALSE TRUE




In [26]:
# Fazendo previsões
previsoes_v2<- predict(modelo_v2,dados_teste)

In [27]:
# Matriz de confusão
print(resultados_v2<-confusionMatrix(previsoes_v2, dados_teste$Revenue))

Confusion Matrix and Statistics

          Reference
Prediction FALSE TRUE
     FALSE  2430  214
     TRUE    172  263
                                          
               Accuracy : 0.8746          
                 95% CI : (0.8624, 0.8861)
    No Information Rate : 0.8451          
    P-Value [Acc > NIR] : 1.836e-06       
                                          
                  Kappa : 0.5034          
                                          
 Mcnemar's Test P-Value : 0.0369          
                                          
            Sensitivity : 0.9339          
            Specificity : 0.5514          
         Pos Pred Value : 0.9191          
         Neg Pred Value : 0.6046          
             Prevalence : 0.8451          
         Detection Rate : 0.7892          
   Detection Prevalence : 0.8587          
      Balanced Accuracy : 0.7426          
                                          
       'Positive' Class : FALSE           
                     

In [28]:
# Score AUC modelo 2
curva_roc2 <- multiclass.roc(response = dados_teste$Revenue, predictor = as.numeric(as.factor(previsoes_v2)))
curva_roc2$auc

Setting direction: controls < cases


Multi-class area under the curve: 0.7426

### Modelo 3

In [31]:
# Construindo modelo 2
modelo_v3 <- svm(Revenue ~ ., data = dados_treino_balanceados,
                kernel = 'linear')

In [32]:
summary(modelo_v3)


Call:
svm(formula = Revenue ~ ., data = dados_treino_balanceados, kernel = "linear")


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  linear 
       cost:  1 

Number of Support Vectors:  38621

 ( 19407 19214 )


Number of Classes:  2 

Levels: 
 FALSE TRUE




In [33]:
# Previsões modelo 3
previsoes_v3<- predict(modelo_v3, dados_teste)

In [34]:
# Matriz de confusão
print(resultados_v3<-confusionMatrix(previsoes_v3, dados_teste$Revenue))

Confusion Matrix and Statistics

          Reference
Prediction FALSE TRUE
     FALSE  2420  217
     TRUE    182  260
                                         
               Accuracy : 0.8704         
                 95% CI : (0.858, 0.8821)
    No Information Rate : 0.8451         
    P-Value [Acc > NIR] : 3.968e-05      
                                         
                  Kappa : 0.4898         
                                         
 Mcnemar's Test P-Value : 0.08873        
                                         
            Sensitivity : 0.9301         
            Specificity : 0.5451         
         Pos Pred Value : 0.9177         
         Neg Pred Value : 0.5882         
             Prevalence : 0.8451         
         Detection Rate : 0.7860         
   Detection Prevalence : 0.8564         
      Balanced Accuracy : 0.7376         
                                         
       'Positive' Class : FALSE          
                                         

In [140]:
# Score AUC modelo 3
curva_roc3 <- multiclass.roc(response = dados_teste$Revenue, predictor = as.numeric(as.factor(previsoes_v3)))
curva_roc3$auc

Setting direction: controls < cases


Multi-class area under the curve: 0.7376

### Modelo 4

In [37]:
# Construindo modelo 4
modelo_v4 <- svm(Revenue ~ ., data = dados_treino_balanceados,
                kernel = 'radial', cost= 2, gamma = 0.01)

In [38]:
# Sumário modelo grid 1
summary(modelo_v4)


Call:
svm(formula = Revenue ~ ., data = dados_treino_balanceados, kernel = "radial", 
    cost = 2, gamma = 0.01)


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  radial 
       cost:  2 

Number of Support Vectors:  31544

 ( 16062 15482 )


Number of Classes:  2 

Levels: 
 FALSE TRUE




In [39]:
# Previsões modelo 4
previsoes_v4 <- predict(modelo_v4, dados_teste)

In [108]:
# Matriz de Confusão modelo 4
print(resultados_v4<-confusionMatrix(previsoes_v4, dados_teste$Revenue))

Confusion Matrix and Statistics

          Reference
Prediction FALSE TRUE
     FALSE  2437  220
     TRUE    165  257
                                          
               Accuracy : 0.875           
                 95% CI : (0.8628, 0.8864)
    No Information Rate : 0.8451          
    P-Value [Acc > NIR] : 1.421e-06       
                                          
                  Kappa : 0.4989          
                                          
 Mcnemar's Test P-Value : 0.005922        
                                          
            Sensitivity : 0.9366          
            Specificity : 0.5388          
         Pos Pred Value : 0.9172          
         Neg Pred Value : 0.6090          
             Prevalence : 0.8451          
         Detection Rate : 0.7915          
   Detection Prevalence : 0.8629          
      Balanced Accuracy : 0.7377          
                                          
       'Positive' Class : FALSE           
                     

In [41]:
# Score AUC modelo 4
curva_roc4 <- multiclass.roc(response = dados_teste$Revenue, predictor = as.numeric(as.factor(previsoes_v4)))
curva_roc4$auc

Setting direction: controls < cases


Multi-class area under the curve: 0.7377

### RESULTADOS

Modelo 1

* Padrão Kernel Radial (RBF)

Modelo 2

* Kernel Radial (RDF) e SMOTE

Modelo 3 

* Kernel Linear e SMOTE

Modelo 4

* Kernel Radial (RDF), SMOTE, Hiper-parâmetro (cost=2,gamma=0.01)

<br>
O modelo 2 com Kernel Radial (RDF) e SMOTE obteve a segunda melhor acurácia 0.8746, mas foi mais equilibrado em sensitivity e specificity, obtendo melhor score AUC.

In [74]:
# Métricas modelo 1
Accuracy<-resultados_v1$overall["Accuracy"]
Curve_auc<-curva_roc1$auc
Sensitivity<-resultados_v1$byClass[1]
Specificity<-resultados_v1$byClass[2]
Precision<-resultados_v1$byClass[5]
Recall<-resultados_v1$byClass[6]
F1<-resultados_v1$byClass[7]

In [73]:
# Métricas modelo 2
Accuracy_v2<-resultados_v2$overall["Accuracy"]
Curve_auc_v2<-curva_roc2$auc
Sensitivity_v2<-resultados_v2$byClass[1]
Specificity_v2<-resultados_v2$byClass[2]
Precision_v2<-resultados_v2$byClass[5]
Recall_v2<-resultados_v2$byClass[6]
F1_v2<-resultados_v2$byClass[7]

In [141]:
# Métricas modelo 3
Accuracy_v3<-resultados_v3$overall["Accuracy"]
Curve_auc_v3<-curva_roc3$auc
Sensitivity_v3<-resultados_v3$byClass[1]
Specificity_v3<-resultados_v3$byClass[2]
Precision_v3<-resultados_v3$byClass[5]
Recall_v3<-resultados_v3$byClass[6]
F1_v3<-resultados_v3$byClass[7]

In [109]:
# Métricas modelo 4
Accuracy_v4<-resultados_v4$overall["Accuracy"]
Curve_auc_v4<-curva_roc4$auc
Sensitivity_v4<-resultados_v4$byClass[1]
Specificity_v4<-resultados_v4$byClass[2]
Precision_v4<-resultados_v4$byClass[5]
Recall_v4<-resultados_v4$byClass[6]
F1_v4<-resultados_v4$byClass[7]

In [150]:
# Métricas
modelo1<- c("Modelo 1",Accuracy,Curve_auc,Sensitivity,Specificity,Precision,Recall,F1)
modelo2<- c("Modelo 2",Accuracy_v2,Curve_auc_v2,Sensitivity_v2,Specificity_v2,Precision_v2,Recall_v2,F1_v2)
modelo3<- c("Modelo 3",Accuracy_v3,Curve_auc_v3,Sensitivity_v3,Specificity_v3,Precision_v3,Recall_v3,F1_v3)
modelo4<- c("Modelo 4",Accuracy_v4,Curve_auc_v4,Sensitivity_v4,Specificity_v4,Precision_v4,Recall_v4,F1_v4)

In [151]:
# Concatenando os resultados
resultado_final<- rbind(modelo1,modelo2,modelo3,modelo4)
rownames(resultado_final)<- c("1","2","3","4")
colnames(resultado_final)<- c("","Accuracy","AUC Score","Sensitivity","Specificity","Precision","Recall","F1")

In [152]:
# Comparando modelos
resultado_final

,Accuracy,AUC Score,Sensitivity,Specificity,Precision,Recall,F1
Modelo 1,0.881779798635921,0.691213580264818,0.967332820906995,0.415094339622642,0.900214592274678,0.967332820906995,0.932567617636162
Modelo 2,0.8746346216304,0.742629842872037,0.933897002305919,0.551362683438155,0.919062027231467,0.933897002305919,0.926420129622569
Modelo 3,0.870412471581682,0.73756359001381,0.930053804765565,0.545073375262054,0.917709518392112,0.930053804765565,0.923840427562512
Modelo 4,0.874959402403378,0.73768565383506,0.936587240584166,0.538784067085954,0.917199849454272,0.936587240584166,0.926792165810991
